<a href="https://colab.research.google.com/github/krusegw/cs315/blob/main/piMonteCarlo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from numba import njit, prange
import datetime
import random
import tensorflow as tf

In [2]:
#Naive approach
n = 10000000
inCircle = 0
start_time = datetime.datetime.now()

for i in range(n):
  x = random.random()
  y = random.random()
  if x**2+y**2<=1.0:
    inCircle +=1

end_time = datetime.datetime.now()
elapsed_time = (end_time - start_time).total_seconds()
pi_est_python = 4.0*inCircle/n
print(pi_est_python,elapsed_time)

3.1427368 4.625354


In [3]:
#Just in time compilation, into machine code
@njit(parallel=True)
def monte_carlo_pi(n):
  inside = 0
  for i in prange(n):
    x = random.random()
    y = random.random()
    if x**2+y**2<=1.0:
      inside +=1
  return inside

n = 10000000
start_time = datetime.datetime.now()
inCircle=monte_carlo_pi(n)
end_time = datetime.datetime.now()
elapsed_time = (end_time - start_time).total_seconds()
pi_est_parallel = 4.0*inCircle/n
print(pi_est_parallel,elapsed_time)

3.1406252 1.47587


In [7]:
# Numpy approach, parallel vectors and hardware optimization, based on https://rse.shef.ac.uk/hpc-intro-tuos-citc/16-parallel/index.html
n = 10000000
#start_time = datetime.datetime.now()

x = np.random.uniform(size=n)
y = np.random.uniform(size=n)
start_time = datetime.datetime.now()
radii = np.sqrt(x*x + y*y)
#counting how many inside the circle
count = len(radii[np.where(radii<=1.0)])

end_time = datetime.datetime.now()
elapsed_time = (end_time - start_time).total_seconds()
pi_est_numpy = 4.0 * count / n

# Since the largest variables in the script are x, y, and radii, each containing n points,
# and each is stored as a NumPy float64, to calculate the total memory required we can
# use NumPy’s dtype function to calculate the size of a float64.
size_of_float = np.dtype(np.float64).itemsize
memory = 3 * n * size_of_float / (1024**3)

print(pi_est_numpy,elapsed_time," memory used, in gigs: ",memory)

3.1413616 0.18724  memory used, in gigs:  0.22351741790771484


In [6]:
# Tensorflow, both hardware and algorithmic optimization
# Convert numpy arrays to TensorFlow tensors
n = 10000000
#start_time = datetime.datetime.now()

x = np.random.uniform(size=n)
y = np.random.uniform(size=n)
x_tf = tf.convert_to_tensor(x, dtype=tf.float32)
y_tf = tf.convert_to_tensor(y, dtype=tf.float32)
start_time = datetime.datetime.now()
radii = np.sqrt(x*x + y*y)
#counting how many inside the circle
count = len(radii[np.where(radii<=1.0)])

end_time = datetime.datetime.now()

elapsed_time = (end_time - start_time).total_seconds()
pi_est_tensorflow = 4.0 * count / n
print(pi_est_tensorflow,elapsed_time)

3.1414652 0.193252
